### Import Required Package

In [331]:
import pandas as pd
import talib
import numpy as np
import json

### Import Tidal

In [332]:
import tidal as td

### Initialize Plumber

In [333]:
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

PLUMBER_HOST = "https://dev-api.ddt-dst.cc/api/plumber/"
with open(f'{str(Path.home())}/.config/gcloud/application_default_credentials.json') as plumber_token:
    token = json.load(plumber_token)

### Load market data and compute technical indicators

In [334]:
import os
import pandas as pd

os.chdir('/home/jovyan/individualized-indicator')
# 定義你想要讀取檔案的目錄
directory = './emb'

start_date = "2022-06-01"
end_date = "2024-09-30"

# 初始化一個空的字典來存儲所有讀取的 DataFrame
dataframes = {}

# 遍歷目錄中的所有檔案
for filename in os.listdir(directory):
    if filename.endswith('.pkl'):
        # 組合完整的檔案路徑
        file_path = os.path.join(directory, filename)
        # 讀取 .pkl 檔案
        df = pd.read_pickle(file_path)
        
        # 使用檔案名稱（不包括副檔名）作為字典的鍵
        variable_name = filename.split('_')[2]
        
        # 將 DataFrame 儲存到字典中
        dataframes[variable_name] = df

keys_list = list(dataframes.keys())
# 將字符串轉換為 datetime 格式，然後進行排序
sorted_dates = sorted(pd.to_datetime(keys_list))
keys_list = [date.strftime('%Y-%m-%d') for date in sorted_dates]

total_df = pd.DataFrame()

for j in range(0,len(keys_list)):
    df_name = keys_list[j]
    selected_df = dataframes[df_name]    
    selected_df = selected_df.reset_index()
    stock_list = pd.DataFrame(selected_df.iloc[:,0].unique())
    total_df = pd.concat([total_df, stock_list], ignore_index=True)
    
unique_values_list = list(total_df.iloc[:,0].unique())
sorted_list = sorted(unique_values_list)

total_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": ",".join([stock for stock in sorted_list]),
    },
)

total_data.index = total_data.index.set_levels(
    pd.to_datetime(total_data.index.levels[1]),level=1)

total_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

total_data['new'] = -1
total_data.reset_index(inplace=True)

def filter_by_year_month_12(filter_data, select_year, select_mon):
    # 計算當前篩選年份和下一個年份
    next_year = select_year + 1
    
    # 構建篩選條件：當前年12月、次年1月、次年2月
    filter_data = filter_data[
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 1)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 2)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 3))
    ]
    
    return filter_data

concat_df = pd.DataFrame()

# for i in range(0,len(keys_list)-1):
for i in range(0,len(keys_list)):
    df_name = keys_list[i]
    selected_df = dataframes[df_name]
    selected_df.reset_index(inplace=True)
    stock_df = selected_df.iloc[:,0]
    stock_df_unique = list(stock_df.unique())
    result = list(set(sorted_list) - set(stock_df_unique))
    filtered_df = total_data[total_data['instrument'].isin(result)]
    filtered_df = filtered_df.rename(columns={'volume': 'vol'})
    selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
    
    select_year = int(df_name.split('-')[0])
    select_mon = int(df_name.split('-')[1])
    select_mons = [(select_mon+1)%12, (select_mon+2)%12, (select_mon+3)%12]
    select_mons = [12 if mon == 0 else mon for mon in select_mons]
    
    selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
    # 過濾出所有6月的交易資料
    filter_data = selected_df[selected_df['datetime'].dt.month.isin(select_mons)]

    # 使用時，根據 select_year 和 select_mon 動態篩選數據
        
    if select_mon == 12:
        filter_data = filter_by_year_month_12(filter_data, select_year, select_mon) 

    else:
        filter_data = filter_data[filter_data['datetime'].dt.year == select_year]
    
    concat_df = pd.concat([concat_df, filter_data], ignore_index=True)
    
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

concat_df.sort_values(by=['instrument', 'datetime'], inplace=True)
concat_df.reset_index(drop=True, inplace=True)
row = np.where(concat_df.iloc[:,0] == '6251')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '8406')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '6548')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
quote_data = concat_df.set_index(['instrument', 'datetime'])
quote_data.columns.values[4] = 'volume'

# quote_data.to_csv('./quote_data.csv')

/tmp/ipykernel_2424/4042907721.py:97: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
/tmp/ipykernel_2424/4042907721.py:97: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
/tmp/ipykernel_2424/4042907721.py:97: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  selected_df['

In [335]:
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

### Strategy Lake API

In [336]:
data_api_url = ("https://dev-api.ddt-dst.cc/api/plumber/nextron/industry_prod_df")
df = pd.read_parquet(
    data_api_url,
    storage_options={
        "gcp-token": json.dumps(token),
    }
)

### Load benchmark data

In [337]:
benchmark_inst = "Y9999"
benchmark_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": benchmark_inst,
    },
)
benchmark_data.index.set_levels(
    pd.to_datetime(benchmark_data.index.levels[1]),
    level=1,
    inplace=True,
)
benchmark_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

benchmark_data

/tmp/ipykernel_2424/1318227356.py:11: FutureWarning: inplace is deprecated and will be removed in a future version.
  benchmark_data.index.set_levels(


open      high       low     close      volume
instrument datetime                                                      
Y9999      2022-06-01  16718.91  16811.04  16649.91  16675.09  5693916000
           2022-06-02  16605.46  16617.26  16540.55  16552.57  4836099000
           2022-06-06  16570.89  16654.12  16509.01  16605.96  5418061000
           2022-06-07  16593.75  16593.75  16465.87  16512.88  4732466000
           2022-06-08  16538.67  16702.99  16538.67  16670.51  4815479000
...                         ...       ...       ...       ...         ...
           2024-09-24  22273.79  22431.78  22154.55  22431.78  7862076000
           2024-09-25  22569.29  22785.32  22569.29  22761.60  9143783000
           2024-09-26  22841.42  22961.77  22769.24  22858.81  9160464000
           2024-09-27  22978.97  23079.21  22806.83  22822.79  9742471000
           2024-09-30  22643.91  22691.23  22224.54  22224.54  9221061000

[569 rows x 5 columns]

### Initialize Tidal
1. Initialize Tidal object
2. Add Quote data (pd.DataFrame)
3. Set strategy object (td.BaseStrategy)
4. Add metric objects (td.BaseMetic)

In [338]:
# Initialize Tidal object
tidal = td.Tidal(init_cash=50000000, slip_ticks=1, stock_config=td.StockConfig.TW, load_configs=True, reqMem="1000Mi", ignore_volume_size=True)

# Add quote data
tidal.add_quote(quote_data)

# Set strategy object
tidal.set_strategy(td.strategy.TopkDropout(5, 1, 'new'))

# Set metric objects
tidal.add_metric(td.metric.AccountInfo())
tidal.add_metric(td.metric.AdditionalInfo())
tidal.add_metric(td.metric.PositionInfo())
tidal.add_metric(td.metric.Portfolio(benchmark_data.loc[benchmark_inst]))

### Stock Config

In [339]:
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.0004275, Min Commission:20.0, Transaction Tax:0.003}

### Instrument Configs

In [340]:
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0}}

### Config Modification

In [341]:
# Set commission to 77% off
tidal.exchange.set_stock_config(commission=0.001425 * 0.23)
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.00032775, Min Commission:20.0, Transaction Tax:0.003}

### Add New Instrument Config

In [342]:
# Add MGC config
tidal.exchange.set_config(instrument='MGC', margin=787., tick_size=0.1, tick_value=1., trade_unit=1, commission=2., min_commission=0., transaction_tax=0.)
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'MGC': InstConfig {Margin:787.0, Tick Size:0.1, Tick Value:1.0, Trade Unit:1, Commission:2.0, Min Commission:0.0, Transaction Tax:0.0}}

### Start Backtesting

In [343]:
tidal.backtest()
logs = tidal.get_logs()
for log in logs:
    print(log)

Tidal Backtesting: 100%|██████████| 548/548 [01:16<00:00,  7.18it/s, cash=6.63e+5, pnl=3.06e+5, position_cost=7.24e+7, value=7.33e+7] 

2024-09-06 - 訂單未能執行: 5340, 可能是由於現金不足或其他原因


### Metric - AccountInfo

In [344]:
account_info = tidal.metrics["AccountInfo"].report
print(account_info)

                    cash    trade_cost  position_cost     slip_cost  \
datetime                                                              
2022-07-01  5.000000e+07  0.000000e+00   0.000000e+00  0.000000e+00   
2022-07-04  5.000000e+07  0.000000e+00   0.000000e+00  0.000000e+00   
2022-07-05  2.115685e+06  1.568894e+04   4.786863e+07  6.085181e+05   
2022-07-06  2.115685e+06  1.568894e+04   4.786863e+07  6.085181e+05   
2022-07-07  6.935143e+05  5.163599e+04   4.948119e+07  5.505211e+05   
...                  ...           ...            ...           ...   
2024-09-24  7.642060e+05  1.703291e+07   7.273263e+07  2.022175e+07   
2024-09-25  5.657524e+05  1.707642e+07   7.233457e+07  2.025499e+07   
2024-09-26  4.979829e+05  1.713222e+07   7.199780e+07  2.055397e+07   
2024-09-27  6.633804e+05  1.718312e+07   7.237495e+07  2.058317e+07   
2024-09-30  6.633804e+05  1.718312e+07   7.237495e+07  2.058317e+07   

                     pnl         value  max_drawdown      turnover  
datetim

### Metric - PositionInfo

In [345]:
position_df = tidal.metrics["PositionInfo"].report
print(position_df)

                       quantity      price   commission    slip_cost  \
instrument datetime                                                    
1102       2022-08-10  258000.0  37.121194  3138.949621  -39165.8964   
           2022-08-11  258000.0  37.121194  3138.949621  -39165.8964   
           2022-08-12  258000.0  37.121194  3138.949621  -39165.8964   
           2022-08-15  258000.0  37.121194  3138.949621  -39165.8964   
           2022-08-17  254000.0  38.524595  3207.114780   53948.4316   
...                         ...        ...          ...          ...   
9945       2024-09-23  326000.0  48.546900  5187.066351 -229210.6000   
           2024-09-25  252000.0  47.795400  3947.565472  112240.8000   
           2024-09-26  252000.0  47.795400  3947.565472  112240.8000   
           2024-09-27  252000.0  47.795400  3947.565472  112240.8000   
           2024-09-30  252000.0  47.795400  3947.565472  112240.8000   

                               pnl  now_price  
instrument date

In [346]:
pi_report = tidal.metrics['PositionInfo'].report
pi_report.iloc[pi_report.index.get_level_values('datetime') == '2022-06-14']

,,quantity,price,commission,slip_cost,pnl,now_price
instrument,datetime,,,,,,


### Metric - AdditionalInfo

In [347]:
tidal.metrics['AdditionalInfo'].report

new
instrument datetime            
1101       2022-07-01  0.000423
           2022-07-04 -0.002125
           2022-07-05  0.000366
           2022-07-06 -0.003043
           2022-07-07  0.001127
...                         ...
9958       2024-09-24  0.000538
           2024-09-25  0.000034
           2024-09-26  0.000216
           2024-09-27  0.000183
           2024-09-30  0.000111

[342085 rows x 1 columns]

### Strategy Lake Submit

In [348]:
group = dict(zip(df["coid"].astype(str), df["tejind4_c"]))
submit_lake_backtest_result = tidal.submit_lake_backtest(account_info=account_info, position_df=position_df, benchmark_info=benchmark_data, group=group,lake_env="dev",strategy_id=19)
print(submit_lake_backtest_result)

update lake backtest data success


### Trade Report

In [349]:
tidal.trade_report

,win_num,lose_num,trade_num,pos_num,win_rate,profit,loss,trade_cost,pnl
instrument,,,,,,,,,
1522,2.0,0.0,2.0,0.0,1.000000,8.941292e+06,0.000000e+00,1.845774e+05,8.941292e+06
2392,2.0,0.0,2.0,0.0,1.000000,7.220887e+06,0.000000e+00,1.248068e+05,7.220887e+06
2059,1.0,1.0,2.0,0.0,0.500000,7.082477e+06,-9.230628e+05,1.481028e+05,6.159415e+06
3029,2.0,0.0,2.0,0.0,1.000000,4.376486e+06,0.000000e+00,8.940971e+04,4.376486e+06
6538,3.0,0.0,3.0,0.0,1.000000,4.331982e+06,0.000000e+00,2.281262e+05,4.331982e+06
...,...,...,...,...,...,...,...,...,...
3022,1.0,2.0,3.0,0.0,0.333333,8.413904e+04,-3.253491e+06,1.570743e+05,-3.169352e+06
4919,2.0,3.0,5.0,1.0,0.400000,7.132841e+05,-3.931905e+06,1.549499e+07,-3.218620e+06
9945,2.0,3.0,5.0,1.0,0.400000,5.263231e+05,-3.774537e+06,1.231467e+07,-3.248214e+06


### Traded instruments

In [350]:
tidal.account.trades.keys()

dict_keys(['2834', '2542', '3552', '6274', '8436', '6683', '8924', '8081', '6712', '2812', '2207', '1102', '6409', '1537', '2640', '6533', '6591', '3551', '5522', '3008', '2409', '6669', '5274', '5434', '2723', '3029', '4104', '6470', '4966', '6605', '6188', '3529', '2458', '2233', '5607', '8086', '8210', '3260', '2474', '3661', '3006', '6290', '1580', '6284', '6176', '2059', '1712', '3038', '6288', '8422', '6139', '4979', '3044', '8048', '4137', '2453', '2916', '6577', '5508', '6691', '4138', '5511', '6538', '6776', '6170', '1522', '8112', '3213', '6523', '3227', '6189', '6585', '5236', '2392', '3338', '9927', '3712', '3622', '4933', '6741', '3624', '6579', '2352', '4919', '5309', '9945', '3022', '6672', '8255', '5340'])

### Trading History

In [351]:
# tidal.account.trades['3015']

### Plot chart by using Plotly

In [352]:
# tidal.analyzer.inst_chart(instrument='3015', metric_name='AdditionalInfo', plot_type=td.PlotType.LINE, scale=1.0)

### Tidal Dashboard

In [353]:
tidal.tdboard()

 * Serving Flask app 'tidal.tdboard'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:39057
 * Running on http://10.136.13.64:39057
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:10.0.10.82 - - [22/Oct/2024 15:47:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [22/Oct/2024 15:47:29] "GET /static/js/main.d754b0a3.js HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [22/Oct/2024 15:47:30] "GET /static/css/main.bf4d504b.css HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [22/Oct/2024 15:47:30] "GET /images/Tidal_Logo_white.png HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [22/Oct/2024 15:47:30] "GET /api/quote/inst_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [22/Oct/2024 15:47:30] "GET /api/metric/metric_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [22/Oct/2024 15:47:30] "GET /api/trade/trade_report HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [22/Oct/2024 15:47:

In [ ]:
tidal.account.position_history